<a href="https://colab.research.google.com/github/HimalKarkal/NFL/blob/Plays-Analysis/Predicting_play_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Aim: Predicting play_result with the help of numeric variables in plays.csv using linear regression and random forest.

In [556]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [557]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/plays.csv')


In [558]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12486 entries, 0 to 12485
Data columns (total 35 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   gameId                            12486 non-null  int64  
 1   playId                            12486 non-null  int64  
 2   ballCarrierId                     12486 non-null  int64  
 3   ballCarrierDisplayName            12486 non-null  object 
 4   playDescription                   12486 non-null  object 
 5   quarter                           12486 non-null  int64  
 6   down                              12486 non-null  int64  
 7   yardsToGo                         12486 non-null  int64  
 8   possessionTeam                    12486 non-null  object 
 9   defensiveTeam                     12486 non-null  object 
 10  yardlineSide                      12319 non-null  object 
 11  yardlineNumber                    12486 non-null  int64  
 12  game

In [559]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12486 entries, 0 to 12485
Data columns (total 35 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   gameId                            12486 non-null  int64  
 1   playId                            12486 non-null  int64  
 2   ballCarrierId                     12486 non-null  int64  
 3   ballCarrierDisplayName            12486 non-null  object 
 4   playDescription                   12486 non-null  object 
 5   quarter                           12486 non-null  int64  
 6   down                              12486 non-null  int64  
 7   yardsToGo                         12486 non-null  int64  
 8   possessionTeam                    12486 non-null  object 
 9   defensiveTeam                     12486 non-null  object 
 10  yardlineSide                      12319 non-null  object 
 11  yardlineNumber                    12486 non-null  int64  
 12  game

In [560]:
df.drop(df.columns[[1,2,3,4,10,15,16,17,22,25,26,27,28,29,30,31,32,33,34]], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12486 entries, 0 to 12485
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   gameId                  12486 non-null  int64  
 1   quarter                 12486 non-null  int64  
 2   down                    12486 non-null  int64  
 3   yardsToGo               12486 non-null  int64  
 4   possessionTeam          12486 non-null  object 
 5   defensiveTeam           12486 non-null  object 
 6   yardlineNumber          12486 non-null  int64  
 7   gameClock               12486 non-null  object 
 8   preSnapHomeScore        12486 non-null  int64  
 9   preSnapVisitorScore     12486 non-null  int64  
 10  prePenaltyPlayResult    12486 non-null  int64  
 11  playResult              12486 non-null  int64  
 12  playNullifiedByPenalty  12486 non-null  object 
 13  absoluteYardlineNumber  12486 non-null  int64  
 14  defendersInTheBox       12481 non-null

In [561]:
df_games = pd.read_csv('/content/drive/My Drive/Colab Notebooks/games.csv')

In [562]:
df = df.merge(df_games[['gameId', 'homeTeamAbbr', 'visitorTeamAbbr']], on='gameId', how='left')

In [563]:
df.head()

,gameId,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,prePenaltyPlayResult,playResult,playNullifiedByPenalty,absoluteYardlineNumber,defendersInTheBox,passProbability,homeTeamAbbr,visitorTeamAbbr
0,2022100908,4,1,10,ATL,TB,41,7:52,21,7,9,9,N,69,7.0,0.747284,TB,ATL
1,2022091103,4,1,10,PIT,CIN,34,7:38,14,20,3,3,N,76,7.0,0.416454,CIN,PIT
2,2022091111,2,2,5,LV,LAC,30,8:57,10,3,15,15,N,40,6.0,0.267933,LAC,LV
3,2022100212,3,2,10,DEN,LV,37,13:12,19,16,7,7,N,47,6.0,0.592704,LV,DEN
4,2022091900,2,1,10,BUF,TEN,35,8:33,7,7,3,3,N,75,7.0,0.470508,BUF,TEN


In [564]:
df['gameClock'] = df['gameClock'].apply(lambda x: round(int(x.split(':')[0]) + int(x.split(':')[1])/60, 2))

In [565]:
df.head()

,gameId,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,prePenaltyPlayResult,playResult,playNullifiedByPenalty,absoluteYardlineNumber,defendersInTheBox,passProbability,homeTeamAbbr,visitorTeamAbbr
0,2022100908,4,1,10,ATL,TB,41,7.87,21,7,9,9,N,69,7.0,0.747284,TB,ATL
1,2022091103,4,1,10,PIT,CIN,34,7.63,14,20,3,3,N,76,7.0,0.416454,CIN,PIT
2,2022091111,2,2,5,LV,LAC,30,8.95,10,3,15,15,N,40,6.0,0.267933,LAC,LV
3,2022100212,3,2,10,DEN,LV,37,13.20,19,16,7,7,N,47,6.0,0.592704,LV,DEN
4,2022091900,2,1,10,BUF,TEN,35,8.55,7,7,3,3,N,75,7.0,0.470508,BUF,TEN


In [566]:
df['possessionTeamScore'] = np.where(df['possessionTeam'] == df['homeTeamAbbr'], df['preSnapHomeScore'], df['preSnapVisitorScore'])
df['defensiveTeamScore'] = np.where(df['defensiveTeam'] == df['homeTeamAbbr'], df['preSnapHomeScore'], df['preSnapVisitorScore'])

In [567]:
df.head()

,gameId,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,prePenaltyPlayResult,playResult,playNullifiedByPenalty,absoluteYardlineNumber,defendersInTheBox,passProbability,homeTeamAbbr,visitorTeamAbbr,possessionTeamScore,defensiveTeamScore
0,2022100908,4,1,10,ATL,TB,41,7.87,21,7,9,9,N,69,7.0,0.747284,TB,ATL,7,21
1,2022091103,4,1,10,PIT,CIN,34,7.63,14,20,3,3,N,76,7.0,0.416454,CIN,PIT,20,14
2,2022091111,2,2,5,LV,LAC,30,8.95,10,3,15,15,N,40,6.0,0.267933,LAC,LV,3,10
3,2022100212,3,2,10,DEN,LV,37,13.20,19,16,7,7,N,47,6.0,0.592704,LV,DEN,16,19
4,2022091900,2,1,10,BUF,TEN,35,8.55,7,7,3,3,N,75,7.0,0.470508,BUF,TEN,7,7


In [568]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12486 entries, 0 to 12485
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   gameId                  12486 non-null  int64  
 1   quarter                 12486 non-null  int64  
 2   down                    12486 non-null  int64  
 3   yardsToGo               12486 non-null  int64  
 4   possessionTeam          12486 non-null  object 
 5   defensiveTeam           12486 non-null  object 
 6   yardlineNumber          12486 non-null  int64  
 7   gameClock               12486 non-null  float64
 8   preSnapHomeScore        12486 non-null  int64  
 9   preSnapVisitorScore     12486 non-null  int64  
 10  prePenaltyPlayResult    12486 non-null  int64  
 11  playResult              12486 non-null  int64  
 12  playNullifiedByPenalty  12486 non-null  object 
 13  absoluteYardlineNumber  12486 non-null  int64  
 14  defendersInTheBox       12481 non-null

In [569]:
df.drop(df.columns[[0,4,5,8,9,10,12,15,16,17]], axis=1, inplace=True)


In [570]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12486 entries, 0 to 12485
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   quarter                 12486 non-null  int64  
 1   down                    12486 non-null  int64  
 2   yardsToGo               12486 non-null  int64  
 3   yardlineNumber          12486 non-null  int64  
 4   gameClock               12486 non-null  float64
 5   playResult              12486 non-null  int64  
 6   absoluteYardlineNumber  12486 non-null  int64  
 7   defendersInTheBox       12481 non-null  float64
 8   possessionTeamScore     12486 non-null  int64  
 9   defensiveTeamScore      12486 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 1.0 MB


In [571]:
df = df[~df['defendersInTheBox'].isnull()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12481 entries, 0 to 12485
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   quarter                 12481 non-null  int64  
 1   down                    12481 non-null  int64  
 2   yardsToGo               12481 non-null  int64  
 3   yardlineNumber          12481 non-null  int64  
 4   gameClock               12481 non-null  float64
 5   playResult              12481 non-null  int64  
 6   absoluteYardlineNumber  12481 non-null  int64  
 7   defendersInTheBox       12481 non-null  float64
 8   possessionTeamScore     12481 non-null  int64  
 9   defensiveTeamScore      12481 non-null  int64  
dtypes: float64(2), int64(8)
memory usage: 1.0 MB


# Data Splitting

In [572]:
#Splitting variables into dependent and independent.

X = df.drop('playResult', axis=1)
y = df['playResult']

In [573]:
#Creating a train/test split.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# ML Models

## Linear Regression

In [574]:
#Training the Model

lr = LinearRegression()
lr.fit(X_train, y_train)


LinearRegression()

In [575]:
#Applying the model to make predictions.

y_lr_train_pred = lr.predict(X_train)
y_lr_test_pred = lr.predict(X_test)

In [576]:
#Evaluating the model

#Training set
lr_train_mse = mean_squared_error(y_train, y_lr_train_pred)
lr_train_r2 = r2_score(y_train, y_lr_train_pred)

#Testing set
lr_test_mse = mean_squared_error(y_test, y_lr_test_pred)
lr_test_r2 = r2_score(y_test, y_lr_test_pred)

#Print Results

print("Training set:")
print('MSE =', lr_train_mse)
print('r2 =', lr_train_r2)
print()
print("Testing set:")
print('MSE =', lr_test_mse)
print('r2 =', lr_test_r2)

Training set:
MSE = 80.89378315819758
r2 = 0.039479927596062336

Testing set:
MSE = 81.55524301682964
r2 = 0.03965066503859194


## Random Forest

In [577]:
#Training the Model

rf = RandomForestRegressor(max_depth = 4, random_state = 42)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=4, random_state=42)

In [578]:
#Applying the model to make predictions

y_rf_train_pred = rf.predict(X_train)
y_rf_test_pred = rf.predict(X_test)

In [579]:
#Evaluating the model

#Training set
rf_train_mse = mean_squared_error(y_train, y_rf_train_pred)
rf_train_r2 = r2_score(y_train, y_rf_train_pred)

#Testing set
rf_test_mse = mean_squared_error(y_test, y_rf_test_pred)
rf_test_r2 = r2_score(y_test, y_rf_test_pred)

#Print Results

print("Training set:")
print('MSE =', rf_train_mse)
print('r2 =', rf_train_r2)
print()
print("Testing set:")
print('MSE =', rf_test_mse)
print('r2 =', rf_test_r2)

Training set:
MSE = 78.89203733901589
r2 = 0.06324834296878112

Testing set:
MSE = 81.26367883478873
r2 = 0.04308396322965691


## Result

In [580]:
results_df = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest'],
    'Training MSE': [lr_train_mse, rf_train_mse],
    'Training R2': [lr_train_r2, rf_train_r2],
    'Testing MSE': [lr_test_mse, rf_test_mse],
    'Testing R2': [lr_test_r2, rf_test_r2]
})

results_df.head()


,Model,Training MSE,Training R2,Testing MSE,Testing R2
0,Linear Regression,80.893783,0.039480,81.555243,0.039651
1,Random Forest,78.892037,0.063248,81.263679,0.043084
